![](Images/p14.png)

![](Images/p15.png)

![](Images/p16.png)

![](Images/p17.png)

![](Images/p18.png)

![](Images/p19.png)

# Python ML Example with Keras
(based on https://elitedatascience.com/keras-tutorial-deep-learning-in-python)

### What we can't offer in this tutorial...

- This is not a complete course on deep learning.
- Instead, this tutorial is meant to get you from zero to your first Convolutional Neural Network with as little headache as possible!
- It demonstrates the power of Python and serves as an use case example for Python.

### Why Keras?

- Keras recommended library for deep learning in Python, especially for beginners. - Its minimalistic, modular approach makes it a breeze to get deep neural networks up and running.
- You can read more about it here:

https://elitedatascience.com/python-deep-learning-libraries#keras

### What is Deep Learning?

- First, we have to talk about neurons, the basic unit of a neural network. 
- A neuron takes inputs, does some math with them, and produces one output. 
- Here’s what a 2-input neuron looks like:

![](Images/neuron.jpeg)

3 things are happening here. First, each input is multiplied by a weight:

$$
\begin{eqnarray}
x_1 &\to& x_1 \times w_1\\
x_2 &\to& x_2 \times w_2
\end{eqnarray}
$$

Next, all the weighted inputs are added together with a bias $b$.

Finally, the sum is passed through an activation function:

The activation function is used to turn an unbounded input into an output that has a nice, predictable form. A commonly used activation function is the sigmoid function:



![](Images/sigmoid.jpeg)

Estimating the $w_i$ is called **training**.

Usually one neuron in one layer is not enough for complex tasks.

- **Deep learning** refers to neural networks with multiple hidden layers that can learn increasingly abstract representations of the input data.
    - This is obviously an oversimplification, but it’s a practical definition for us right now.

- For example, deep learning has led to major advances in computer vision.
- We’re now able to classify images, find objects in them, and even label them with captions.
- To do so, deep neural networks with many hidden layers can sequentially learn more complex features from the raw input image:
    - The first hidden layers might only learn local edge patterns.
    - Then, each subsequent layer (or filter) learns more complex representations.
    - Finally, the last layer can classify the image as a cat or kangaroo.
- These types of deep neural networks are called **Convolutional Neural Networks**.

![](Images/neural_net2.jpeg)

### What are Convolutional Neural Networks?

- Convolutional Neural Networks (CNN’s) are multi-layer neural networks (sometimes up to 17 or more layers) that assume the input data to be images.

![](Images/neural_net2.jpeg)

![](Images/typical_cnn_architecture.png)

- By making this requirement, CNN’s can drastically reduce the number of parameters that need to be tuned.
- Therefore, CNN’s can efficiently handle the high dimensionality of raw images.

- Their underlying mechanics are beyond the scope of this tutorial.
  (https://cs231n.github.io/convolutional-networks/)

# 8 Steps towards a first CNN

### Step 1: Import libraries

- Let’s start by importing matplotlib and numpy. 
- Then set a seed for the computer’s pseudorandom number generator. 

In [ ]:
import matplotlib
import numpy as np
np.random.seed(123)

- Next, we’ll import the Sequential model type from Keras.
- This is simply a linear stack of neural network layers.
- It’s perfect for the type of feed-forward CNN we’re building in this tutorial.

In [ ]:
from keras.models import Sequential

- Next, let’s import the “core” layers from Keras.
- These are the layers that are used in almost any neural network:

In [ ]:
from keras.layers import Dense, Dropout, Activation, Flatten

- Then, we’ll import the CNN layers from Keras.
- These are the convolutional layers that will help us efficiently train on image data:

In [ ]:
from keras.layers import Convolution2D, MaxPooling2D

- Finally, we’ll import some utilities. This will help us transform our data later:

In [ ]:
from keras.utils import np_utils

### Step 2: Load image data from MNIST.

- MNIST is a great dataset for getting started with deep learning and computer vision.
- It’s a big enough challenge to warrant neural networks, but it’s manageable on a single computer.
- That makes it perfect for this Keras tutorial.

- The Keras library conveniently includes it already. We can load it like so:

In [ ]:
from keras.datasets import mnist
 
# Load pre-shuffled MNIST data into train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


- What shape does the dataset have?

In [ ]:
print( X_train.shape )

- 60000 entries 
- each one is a 28x28 pixel picture
- Let's have a look at a few:

In [ ]:
from matplotlib import pyplot as plt
plt.imshow(X_train[0])

In [ ]:
plt.imshow(X_train[1])

In [ ]:
plt.imshow(X_train[2])

In [ ]:
plt.imshow(X_train[3])

In [ ]:
plt.imshow(X_train[4])

### Step 3: Preprocess input data for Keras.


- When using the TensorFlow backend, you must explicitly declare a dimension for the number of channels in the input images. 
    - For example, a full-color image with all 3 RGB channels will have a channel value of 3.

- Our MNIST images only have 1 channel, but we must explicitly declare that.

- **We want to transform our dataset from having shape (n, width, height) to (n, width, height, channels).**

In [ ]:
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)

- To confirm, we can print X_train’s dimensions again:

In [ ]:
print( X_train.shape )
# (60000, 28, 28, 1)

- The final preprocessing step for the input data is to convert our data type to float32 and normalize our data values to the range [0, 1].

In [ ]:
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

Now, our input data are ready for model training.

### Step 4: Preprocess class labels for Keras

- Class labels are basically the "true values".
- Let’s take a look at the shape of our class label data:

In [ ]:
print( y_train.shape )
# (60000,)

In [ ]:
y_train[0], y_train[1], y_train[2], y_train[3], y_train[4], 

- We should have 10 different classes, one for each digit. 
- But it looks like we only have a 1-dimensional array.

- This can be easily fixed:

In [ ]:
# Convert 1-dimensional class arrays to 10-dimensional class matrices
Y_train = np_utils.to_categorical(y_train, 10)
Y_test = np_utils.to_categorical(y_test, 10)

- Now we can take another look:

In [ ]:
print( Y_train.shape )
# (60000, 10)

- That looks much better.

### Step 5: Define model architecture

- Now we’re ready to define our model architecture. 
    - How many layers? What kind of layers? ...
- In actual R&D work, researchers will spend a considerable amount of time studying model architectures.

- We don't have the time to re-do the architecture analysis.
- We'll provide the actual architecture.

- Let’s start by declaring a sequential model format:

In [ ]:
model = Sequential()

- Next, we declare the input layer:

In [ ]:
model.add(Convolution2D(32, (3,3), activation='relu', input_shape=(28,28,1)))


- The input shape parameter should be the shape of 1 sample. 
- In this case, it’s the same (28, 28, 1) that corresponds to the (width, height, channels) of each digit image.

- But what do the first two parameters represent? 
- They correspond to the number of convolution filters to use (32) and the number of rows and columns (3, 3) in each convolution kernel.

    - Note: The step size is (1,1) by default, and it can be tuned using the ‘strides‘ parameter.

- We can confirm this by printing the shape of the current model output:

In [ ]:
print( model.output_shape )
# (None, 26, 26, 32)

- That output corresponds to (samples, new_rows, new_cols, filters). 
- In other words, the current model will output all of the samples, convoluted into a 26×26 array using 32 filters.
---
- Next, we can simply add more layers to our model like we’re building legos:

In [ ]:
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

- Again, we won’t go into the theory too much, but it’s important to highlight the Dropout layer we just added: 
    - This is a method for regularizing our model in order to prevent **overfitting**.

![](Images/dropout.jpeg)

![](Images/overfitting.jpeg)

- '''MaxPooling2D''' is a way to reduce the number of parameters in our model by sliding a 2×2 pooling filter across the previous layer and taking the max of the 4 values in the 2×2 filter.

- So far, for model parameters, we’ve added two Convolution layers. 
- To complete our model architecture, let’s add a **fully connected** layer and then the output layer:

In [ ]:
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(10, activation='softmax'))

- For **Dense layers**, the first parameter is the output size of the layer. Keras automatically handles the connections between layers.

- Note that the final layer has an output size of 10, corresponding to the 10 classes of digits.

- Also note that the weights from the Convolution layers must be flattened (made 1-dimensional) before passing them to the fully connected Dense layer.
- ReLU is defined as
$$
\mathrm{ReLU}(x) = \max(0,x)
$$
- Softmax is defined as

$$
\mathrm{softmax}(x_i) = \dfrac{\exp{x_i}}{\sum_j \exp{x_j}}\in[0..1]
$$

---

- Now all we need to do is define the loss function and the optimizer, and then we’ll be ready to train it.

### Step 6: Compile model

- Almost done!
- We just need to compile the model and we’ll be ready to train it. 
- When we **compile** the model, we declare the loss function and the optimizer (SGD, Adam, etc.).
- N.B. Keras has a variety of loss functions and out-of-the-box optimizers to choose from.

In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

### Step 7: Fit model on training data.

- To fit the model, all we have to do is declare the batch size and number of epochs to train for, then pass in our training data.

In [ ]:
train = model.fit(X_train, Y_train, 
          batch_size=32, epochs=2, verbose=1) # epochs 10

### Step 8: Evaluate model on test data.


- Finally, we can evaluate our model on the test data:


In [ ]:
score = model.evaluate(X_test, Y_test, verbose=0)

In [ ]:
print('Test loss:', score[0]) 
print('Test accuracy:', score[1])

In [ ]:
plt.plot(train.history['loss'], label='train')
plt.title('Model Loss')
plt.xlabel('epochs')
plt.ylabel('loss values')
plt.legend(loc='upper right')
plt.show()

In [ ]:
plt.plot(train.history['accuracy'], label='test')
plt.title('Model Accuracy')
plt.xlabel('epochs')
plt.ylabel('accuracy values')
plt.legend(loc='upper right')
plt.show()

#### Model Loss
![](Images/loss.png)

#### Model Accuracy
![](Images/Accuracy.png)